In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import pandas as pd
import numpy as np  
import re
import os

from tensorflow import keras
from tensorflow.keras.models import load_model
import tensorflow.python.keras.backend as K
# from tensorflow.keras import backend as K
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers

from keras_bert import load_trained_model_from_checkpoint
from keras_bert import Tokenizer
from keras_bert import AdamWarmup, calc_train_steps
from keras_radam import RAdam

import codecs

Instructions for updating:
non-resource variables are not supported in the long term


In [28]:
import googletrans 
from langdetect import detect
translator = googletrans.Translator()

In [3]:
SEQ_LEN = 128
BATCH_SIZE = 16
EPOCHS=2
LR=1e-5

path ="model\\bert"
config_path = os.path.join(path, 'bert_config.json')
checkpoint_path = os.path.join(path, 'bert_model.ckpt')
vocab_path = os.path.join(path, 'vocab.txt')

DATA_COLUMN = "document"
LABEL_COLUMN = "label"

In [4]:
token_dict = {}
with codecs.open(vocab_path, 'r', 'utf8') as reader:
    for line in reader:
        token = line.strip()
        if "_" in token:
          token = token.replace("_","")
          token = "##" + token
        token_dict[token] = len(token_dict)

In [5]:
class inherit_Tokenizer(Tokenizer):
    def _tokenize(self, text):
        if not self._cased:
            text = text

            text = text.lower()
        spaced = ''
        for ch in text:
            if self._is_punctuation(ch) or self._is_cjk_character(ch):
                spaced += ' ' + ch + ' '
            elif self._is_space(ch):
                spaced += ' '
            elif ord(ch) == 0 or ord(ch) == 0xfffd or self._is_control(ch):
                continue
            else:
                spaced += ch
        tokens = []
        for word in spaced.strip().split():
            tokens += self._word_piece_tokenize(word)
        return tokens

In [6]:
tokenizer = inherit_Tokenizer(token_dict)

In [7]:
layer_num = 12
model = load_trained_model_from_checkpoint(
    config_path,
    checkpoint_path,
    training=True,
    trainable=True,
    seq_len=SEQ_LEN,)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [8]:
def get_bert_finetuning_model(model):
    inputs = model.inputs[:2]
    dense = model.layers[-3].output


    outputs = keras.layers.Dense(1, activation='sigmoid',kernel_initializer=keras.initializers.TruncatedNormal(stddev=0.02),
                              name = 'real_output')(dense)



    bert_model = keras.models.Model(inputs, outputs)
    bert_model.compile(
      optimizer=RAdam(learning_rate=0.00001, weight_decay=0.0025),
      loss='binary_crossentropy',
      metrics=['accuracy'])

    return bert_model

In [9]:
bert_model = get_bert_finetuning_model(model)
bert_model.load_weights(path+"/bert.h5")

In [10]:
def sentence_convert_data(data):
    global tokenizer
    indices = []
    ids, segments = tokenizer.encode(data, max_len=SEQ_LEN)
    indices.append(ids)
        
    items = indices
    indices = np.array(indices)
    return [indices, np.zeros_like(indices)]

def movie_evaluation_predict(sentence):
    data_x = sentence_convert_data(sentence)
    predict = bert_model.predict(data_x)
    predict_answer = np.round(np.ravel(predict), 0).item()
    
    return predict_answer
#     if predict_answer == 0:
#         print("부정적인 평가입니다.")
#     elif predict_answer == 1:
#         print("긍정적인 평가입니다.")

In [29]:
def detect_func(s):
    try:
        return detect(s)
    except:
        pass

In [11]:
movie_evaluation_predict(
"""

""")

1.0

In [12]:
how_we_met = pd.read_excel("data/how_we_met1.xlsx", engine='openpyxl')

In [13]:
prank = pd.read_excel("data/prank1.xlsx", engine='openpyxl')

In [14]:
russian_soldier = pd.read_excel("data/russian_soldiers.xlsx", engine='openpyxl')

In [15]:
how_we_met['comment'] = how_we_met['comment'].str.replace('<br>','')

In [16]:
how_we_met

,comment,author,date,num_likes
0,자막이 겹쳐보인다면 설정에서 자막을 끄실 수 있습니다!^^ por favor act...,의대생과 공대생 [Kris and ME],2020-08-21T21:16:20Z,32
1,"Нифига )))) вам везет Скучаю по России ..Кст, ...",아라사의추위,2021-07-20T11:08:11Z,0
2,이런 순수한 고백.사랑...경험해본지 30년은 넘은듯 한데... 영상을 보는내내 ...,인생은추억,2021-04-28T11:35:23Z,1
3,용기있는자가 미인을 얻는다는말..,chul7656,2021-04-22T07:19:06Z,0
4,세번이나 차이고 사귀다뉘!!!난 한번차인 순간 ...안뇨오오오오옹~~~,우리집,2021-04-07T10:39:20Z,0
...,...,...,...,...
163,영상보는 동안 나의 입가에 미소를 지었어요 .두분 그사랑의 끈을 항상 잡고 있으세요,SeongGyu Kim,2020-08-21T20:59:30Z,42
164,크리스를 보면 때묻지않은 순수함이 느껴져서 좋네요^^맨얼굴 츄리닝 티셔츠 운동화만...,R W,2020-08-21T20:58:15Z,74
165,any picture of Kris looking more like a boy? :-),Lil' Sun,2020-08-21T20:57:27Z,3
166,Thanks for sharing the story. It&#39;s lovely.,Sangwoo Im,2020-08-21T20:43:17Z,16


In [18]:
prank['comment'] = prank['comment'].str.replace('<br>','')

In [19]:
russian_soldier['comment'] = russian_soldier['comment'].str.replace('<br>','')

In [36]:
how_we_met['lang'] = how_we_met['comment'].apply(detect_func)

In [21]:
# for i in range(len(how_we_met)):
#     how_we_met.loc[i, 'lang'] = translator.detect(how_we_met.iloc[i]['comment']).lang

In [30]:
prank['lang'] = prank['comment'].apply(detect_func)

In [31]:
prank

,comment,author,date,num_likes,lang
0,이 영상으로 불편하셨다면 죄송합니다 ㅠ 앞으로 이런식의 몰카는 하지 않겠습니다..!...,의대생과 공대생 [Kris and ME],2020-06-22T13:37:26Z,212,en
1,The love that Kris has for you is enormous. (w...,199515Danny,2020-11-12T00:19:36Z,0,en
2,쑤레기짓하지맙시다,와이낫,2020-09-04T05:25:43Z,0,ko
3,이 영상의 교훈..연인간의 몰카 잘못찍다가..사단날수 있다..몰카도 적당한걸로 ㅋㅋ,파랑,2020-08-30T22:26:38Z,0,ko
4,여친이 너무 사랑해서 몰카도 힘든 수준이군요 ㅋㅋㅋㅋ 그냥 너무 부럽습니다,칼바람,2020-08-10T05:07:45Z,0,ko
...,...,...,...,...,...
747,네 적성에 안맞는거 같아요 ㅠㅠㅋㅋㅋ,의대생과 공대생 [Kris and ME],2020-06-22T11:14:09Z,1,ko
748,눈물흘릴때 너무 귀엽기도 하면서 내가 더 미안하네 ㅠㅠ,May Ham,2020-06-22T11:08:20Z,34,ko
749,저도 너무 미안했어요..ㅠㅠ,의대생과 공대생 [Kris and ME],2020-06-22T11:14:41Z,3,ko
750,와 진짜 서로 사랑하는게 많이 느껴진다 크리스 눈물 흘릴때 너무 짠했음 ㅠㅠㅠ 크리...,0 mm,2020-06-22T11:07:13Z,63,ko


In [32]:
# for i in range(len(prank)):
#     prank.loc[i, 'lang'] = translator.detect(prank.loc[i, 'comment']).lang

In [33]:
russian_soldier['lang'] = russian_soldier['comment'].apply(detect_func)

In [37]:
# for i in range(len(russian_soldier)):
#     russian_soldier.loc[i, 'lang'] = translator.detect(russian_soldier.loc[i, 'comment']).lang

In [38]:
how_we_met.loc[how_we_met['lang']!='ko', 'comment'] = how_we_met[how_we_met['lang']!='ko']['comment'].map(lambda x: translator.translate(x, dest='ko').text)

In [40]:
# prank.loc[prank['lang']!='ko', 'comment'] = prank.loc[prank['lang']!='ko', 'comment'].apply(translator.translate, dest='ko').text

In [41]:
prank.loc[prank['lang']!='ko', 'comment'] = prank.loc[prank['lang']!='ko', 'comment'].map(lambda x: translator.translate(x, dest='ko').text)

In [42]:
russian_soldier.loc[russian_soldier['lang']!='ko', 'comment'] = russian_soldier.loc[russian_soldier['lang']!='ko', 'comment'].map(lambda x: translator.translate(x, dest='ko').text)

In [43]:
def evaluate(df):
    pos = 0
    neg = 0
    
    for i in range(len(df)):
        if movie_evaluation_predict(df.loc[i, 'comment']) == 1:
            pos += 1
        else:
            neg += 1
    
    print(f'positive comments : {pos} negative comments : {neg} positive comments ratio : {pos / (pos + neg) * 100 :.2f}')    
    
    return pos, neg

In [69]:
def evaluate_by_country(df):
    country = ['ko', 'en', 'ru']
    
    for c in country:
        pos = 0
        neg = 0
        df_tmp = df[df['lang']==c]
        for _, row in df_tmp.iterrows():
            if movie_evaluation_predict(row['comment']) == 1:
                pos += 1
            else:
                neg += 1
        print(f'{c} lang positive comments : {pos} negative comments : {neg} positive comments ratio : {pos / (pos + neg) * 100 :.2f}')    

In [70]:
evaluate_by_country(how_we_met)

ko lang positive comments : 103 negative comments : 26 positive comments ratio : 79.84
en lang positive comments : 21 negative comments : 3 positive comments ratio : 87.50
ru lang positive comments : 7 negative comments : 0 positive comments ratio : 100.00


In [45]:
evaluate(how_we_met)

positive comments : 137 negative comments : 31 positive comments ratio : 81.55


(137, 31)

In [72]:
evaluate_by_country(how_we_met)

ko lang positive comments : 103 negative comments : 26 positive comments ratio : 79.84
en lang positive comments : 21 negative comments : 3 positive comments ratio : 87.50
ru lang positive comments : 7 negative comments : 0 positive comments ratio : 100.00


In [47]:
evaluate(prank)

positive comments : 508 negative comments : 244 positive comments ratio : 67.55


(508, 244)

In [73]:
evaluate_by_country(prank)

ko lang positive comments : 411 negative comments : 224 positive comments ratio : 64.72
en lang positive comments : 59 negative comments : 8 positive comments ratio : 88.06
ru lang positive comments : 16 negative comments : 2 positive comments ratio : 88.89


In [49]:
evaluate(russian_soldier)

positive comments : 269 negative comments : 92 positive comments ratio : 74.52


(269, 92)

In [74]:
evaluate_by_country(russian_soldier)

ko lang positive comments : 216 negative comments : 81 positive comments ratio : 72.73
en lang positive comments : 25 negative comments : 4 positive comments ratio : 86.21
ru lang positive comments : 15 negative comments : 2 positive comments ratio : 88.24
